In [1]:
import cv2
import numpy as np
import urllib.request as urlreq
import os
import matplotlib.pyplot as plt
from pylab import rcParams
from pathlib import Path
import face_alignment
from skimage import io


Bad key "text.kerning_factor" on line 4 in
d:\new\copied_from_c_tmp\anaconda\envs\dm\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
video_input_file = Path('data/orlando_bloom.avi')
video_output_file = Path('data/orlando_bloom_output.mp4')

In [4]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)

In [6]:
cap = cv2.VideoCapture(str(video_input_file))
# cap = cv2.VideoCapture(0)

width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH )), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT ))
fps = int(cap.get(cv2.CAP_PROP_FPS))
prop_fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
prop_format = int(cap.get(cv2.CAP_PROP_FORMAT))

# print(f'width {width}, height {height}, fps {fps}, fourcc {prop_fourcc}, format {prop_format}')
print(f'frames_count {frames_count}')

# Play input video

# while(cap.isOpened()):
#     ret, frame = cap.read()

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
# #     pose_analyzed = analyze_pose(frame)
    
#     cv2.imshow('frame',gray)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

cap.release()
# cv2.destroyAllWindows()

frames_count 139


In [9]:
def analyze_pose(im):
    size = im.shape
    preds = fa.get_landmarks(im)
    
    landmarks = preds
    im_landmarks = im.copy()
    i = 0
    for landmark in landmarks:
        for x,y in landmark:
            if i == 8 or i == 30 or i == 36 or i == 45 or i == 48 or i == 54:
                # with white colour in BGR and thickness 1
                cv2.circle(im_landmarks, (x, y), 1, (255, 255, 255), 2)
            i += 1
    
    image_points = np.array([
                            landmarks[0][30],     # Nose tip
                            landmarks[0][8],     # Chin
                            landmarks[0][36],     # Left eye left corner
                            landmarks[0][45],     # Right eye right corne
                            landmarks[0][48],     # Left Mouth corner
                            landmarks[0][54]      # Right mouth corner
                        ], dtype="double")
    
    # 3D model points.
    model_points = np.array([
                                (0.0, 0.0, 0.0),             # Nose tip
                                (0.0, -330.0, -65.0),        # Chin
                                (-225.0, 170.0, -135.0),     # Left eye left corner
                                (225.0, 170.0, -135.0),      # Right eye right corne
                                (-150.0, -150.0, -125.0),    # Left Mouth corner
                                (150.0, -150.0, -125.0)      # Right mouth corner

                            ])


    # Camera internals

    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

    
    (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)

    for p in image_points:
        cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)


    p1 = ( int(image_points[0][0]), int(image_points[0][1]))
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))

    cv2.line(im, p1, p2, (255,0,0), 2)

    return im


In [10]:
# Estimate pose and save file

cap = cv2.VideoCapture(video_input_file)
# Uncomment line (1) and lines (3) if video from a web camera is to be used
# cap = cv2.VideoCapture(0)       # line (1)

# Define the codec and create VideoWriter object
out = cv2.VideoWriter(video_output_file, prop_fourcc, fps, (width, height))

i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:  
        if (i / fps) * 100 % 10 == 0:
            print(f'{i}% complete')
        frame = analyze_pose(frame)
        
#         cv2.imshow('frame',frame)                #lines (3)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
            
        out.write(frame)
    else:
        break
    i += 1

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

0
10
20
30
40
50
60
70
80
90
100
110
120
130


In [15]:
# Play result
cap = cv2.VideoCapture(video_output_file)

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()